# Power Method

In [1]:
import numpy as np
def power_method_find(A :list,x0: list,tol = 1e-6):
    '''
    # Power Method
    This function finds the largest eigenvalue and the corresponding eigenvector

    ## Condition
    - n x n matrix A has n linearly independent eigenvectors
    - Eigenvalues can be ordered in magnitude : |λ1| > |λ2| > · · · > |λn|. The λ1 is called the dominant eigenvalue and the corresponding eigenvector is the dominant eigenvector of A.

    ## Paremeters
    - A: The matrix for which the eigenvalues and eigenvectors are to be found
    - x0: The initial guess for the eigenvector
    - tol: The tolerance for the solution
    ## Returns
    - eigval: The largest eigenvalue
    - eigvec: The corresponding eigenvector
    '''
    A=np.array(A)
    x0=np.array(x0)
    x_copy = np.copy(x0)
    lam_0 = np.matmul(np.matmul(np.linalg.matrix_power(A,2),x0).T,np.matmul(np.linalg.matrix_power(A,1),x0))[0][0]/np.matmul(np.matmul(np.linalg.matrix_power(A,1),x0).T,np.matmul(np.linalg.matrix_power(A,1),x0))[0][0]
    lam_1 = np.matmul(np.matmul(np.linalg.matrix_power(A,3),x0).T,np.matmul(np.linalg.matrix_power(A,2),x0))[0][0]/np.matmul(np.matmul(np.linalg.matrix_power(A,2),x0).T,np.matmul(np.linalg.matrix_power(A,2),x0))[0][0]
    i=3
    while abs(lam_1-lam_0)>tol:
        lam_0 = lam_1
        lam_1 = np.matmul(np.matmul(np.linalg.matrix_power(A,i+1),x0).T,np.matmul(np.linalg.matrix_power(A,i),x0))[0][0]/np.matmul(np.matmul(np.linalg.matrix_power(A,i),x0).T,np.matmul(np.linalg.matrix_power(A,i),x0))[0][0]
        i+=1

    eigval = lam_1
    eigvec = np.matmul(np.linalg.matrix_power(A,i-1),x_copy)
    norm = np.linalg.norm(eigvec)
    eigvec = eigvec/norm
    return eigval,eigvec,i    


A = np.array([
    [4,    2/3, -4/3, 4/3],
    [2/3,   4,   0,    0],
    [-4/3,  0,   6,    2],
    [4/3,   0,   2,    6]
    ])

x0 =np.array([[4],[3],[7],[9]])
val,vec = np.linalg.eigh(A)
print(val)
print(power_method_find(A,x0))



[2. 4. 6. 8.]
(7.999999002873864, array([[4.99281015e-04],
       [1.66473835e-04],
       [7.06773763e-01],
       [7.07439447e-01]]), 23)


In [3]:
import numpy as np
A = np.ones(2)
A

array([1., 1.])

# Linear fitting

In [46]:
def linear_fit(xlist: list,ylist: list,elist: list):
    '''
    # Linear Regression
    This function finds the best fit line for a given set of data points
    Finds the fit for the equation y = a + bx
    ## Parameters
    - xlist: The x-coordinates of the data points
    - ylist: The y-coordinates of the data points
    - elist: The error in the y-coordinates of the data points. If elist=False, the function will assume that the error is 1 for all data points
    ## Returns
    - slope: The slope of the best fit line
    - intercept: The y-intercept of the best fit line
    - chi_sq: The chi-squared value of the best fit line
    '''
    # Raise an error if the lengths of xlist, ylist, and elist are not the same
    if len(xlist) != len(ylist):
        raise ValueError('The length of xlist, ylist, and elist must be the same')
    
    # If elist is False, assume that the error is 1 for all data points
    if elist == False:
        elist = [1]*len(xlist)
    # Convert the lists to numpy arrays
    xlist = np.array(xlist)
    ylist = np.array(ylist)
    elist = np.array(elist)
    n=len(xlist)
    # Calculate the sums
    S=np.sum(1/((elist)**2))
    Sx = np.sum(xlist/((elist)**2))
    Sy = np.sum(ylist/((elist)**2))
    Sxx = np.sum((xlist**2)/((elist)**2))
    Syy = np.sum((ylist**2)/((elist)**2))
    Sxy = np.sum((xlist*ylist)/((elist)**2))

    # Calculate the slope and intercept
    Delta = S*Sxx - Sx**2

    intercept=(Sxx*Sy-Sx*Sxy)/Delta
    slope=(S*Sxy-Sx*Sy)/Delta
    # Calculate the error in the slope and intercept
    # error_intercept = np.sqrt(Sxx/Delta)
    # error_slope = np.sqrt(S/Delta)
    # cov = -Sx/Delta
    # Pearsen's correlation coefficient
    r_sq = Sxy/np.sqrt(Sxx*Syy) 

    return slope,intercept,np.sqrt(r_sq)




xlist = [1,2,3,4,5]
ylist = [2,4,6,8,10]
elist = [1,1,1,1,1]
linear_fit(xlist,ylist,elist)

(2.0, 0.0, 1.0)

# Polynomial fit


In [65]:
import numpy as np
from Library_Classworks import Gauss_seidel_solve,gauss_jordan_solve

def polynomial_fit(xlist: list,ylist: list,sigma_list: list,degree: int,tol=1e-6):
    '''
    # Polynomial Fitting
    This function finds the best fit polynomial for a given set of data points
    Finds the fit for the equation y = a0 + a1*x + a2*x^2 + ... + an*x^n
    ## Parameters
    - xlist: The x-coordinates of the data points
    - ylist: The y-coordinates of the data points
    - sigma_list: The error in the y-coordinates of the data points
    - degree: The degree of the polynomial to be fit
    ## Returns
    - a: The coefficients of the best fit polynomial
    '''
    xlist = np.array(xlist)
    ylist = np.array(ylist)
    sigma_list = np.array(sigma_list)
    A_matrix = np.zeros((degree+1,degree+1))

    for i in range(degree+1):
        for j in range(degree+1):
            A_matrix[i][j] = np.sum((xlist**(i+j))/(sigma_list**2))
    B_matrix = np.zeros(degree+1)
    for i in range(degree+1):
        B_matrix[i] = np.sum((ylist*(xlist**i))/(sigma_list**2))
    a = np.linalg.solve(A_matrix,B_matrix)
    return a


xlist = [1,2,3,4,5]
ylist = [1,4,9,16,25]
sigma_list = [1,1,1,1,1]
degree = 2
polynomial_fit(xlist,ylist,sigma_list,degree)


array([ 0., -0.,  1.])

(0.28366218546322625-0.9589242746631385j)